In [2]:
using Graphs, Graphs.Experimental.ShortestPaths
using GraphIO

In [3]:
using GraphIO.NET
using Statistics
using Pandas

file_path = "A1-networks/toy/wheel.net"
graph = loadgraph(file_path, "graph_key", NETFormat())

{9, 16} undirected simple Int64 graph

In [113]:
function read_weighted_net(file_path)
    graph = loadgraph(file_path, "graph_key", NETFormat())
    
    edge_weights = Dict{Tuple{Int,Int}, Float64}()
    open(file_path, "r") do file
        reading_edges = false
        for line in eachline(file)
            if occursin("*Edges", line)
                reading_edges = true
            elseif reading_edges
                edge_data = split(line)
                if length(edge_data) >= 3
                    src, dst, weight = parse(Int, edge_data[1]), parse(Int, edge_data[2]), parse(Float64, edge_data[3])
                    edge_weights[(src, dst)] = weight
                    # Assuming you add edges to your graph here
                    add_edge!(graph, src, dst)  # Add the edge to the graph if not already added
                end
            end
        end
    end
    return graph, edge_weights
end

read_weighted_net (generic function with 1 method)

In [114]:
graph, edge_weights = read_weighted_net("A1-networks/toy/wheel.net")

(SimpleGraph{Int64}(16, [[2, 3, 4, 5, 6, 7, 8, 9], [1, 3, 9], [1, 2, 4], [1, 3, 5], [1, 4, 6], [1, 5, 7], [1, 6, 8], [1, 7, 9], [1, 2, 8]]), Dict((4, 5) => 1.0, (1, 2) => 1.0, (1, 3) => 1.0, (1, 4) => 1.0, (9, 2) => 1.0, (7, 8) => 1.0, (8, 9) => 1.0, (1, 9) => 1.0, (3, 4) => 1.0, (5, 6) => 1.0…))

In [106]:
function network_num_descriptors(graph)
    # Number of nodes and edges
    num_nodes = nv(graph)
    num_edges = ne(graph)

    # Degrees
    degrees = degree(graph)
    min_degree = minimum(degrees)
    max_degree = maximum(degrees)
    avg_degree = mean(degrees)

    # Average clustering coefficient 
    avg_clustering_coefficient = sum(local_clustering_coefficient(graph, vertices(graph)))/nv(graph)

    # Assortativity 
    assort = assortativity(graph)

    # Average path length and Diameter
    # For a large graph, this might be computationally expensive
    avg_path_length = sum(shortest_paths(graph).dists)/(num_nodes*(num_nodes-1))
    diam = Graphs.diameter(graph)  # Same adjustment as above

    # Printing the results
    println("Number of nodes: $num_nodes")
    println("Number of edges: $num_edges")
    println("Degree -- Min: $min_degree, Max: $max_degree, Avg: $avg_degree")
    println("Average Clustering Coefficient: $avg_clustering_coefficient")
    println("Assortativity: $assort")
    println("Average Path Length: $avg_path_length")
    println("Diameter: $diam")
end

network_num_descriptors (generic function with 1 method)

In [107]:
network_num_descriptors(graph)

Number of nodes: 9
Number of edges: 16
Degree -- Min: 3, Max: 8, Avg: 3.5555555555555554
Average Clustering Coefficient: 0.6243386243386244
Assortativity: -0.3333333333333333
Average Path Length: 1.5555555555555556
Diameter: 2


In [ ]:
function construct_df(data_dict, col_names = [])
    # Initialize an empty DataFrame with appropriate column names
    # Assuming all vectors are of the same length and represent the same variables
    if col_names == []
        col_names = ["Col$(i)" for i in 1:length(first(values(data_dict)))]
    end
    
    df = DataFrames.DataFrame(; (Symbol(col_name) => Float64[] for col_name in col_names)...)

    # Sort the dictionary by keys to maintain the order
    sorted_keys = sort(collect(keys(data_dict)))

    # Populate the DataFrame
    for key in sorted_keys
        push!(df, data_dict[key])
    end
    return df
end

In [ ]:
function compute_strength(edge_weights, mode="undirected")
    weights_count = Dict{Int, Int}()
    if mode == "undirected"
        for (key,value) in edge_weights
            for v in key
                if haskey(weights_count, v)
                    weights_count[v] += value
                else
                    weights_count[v] = value
                end
            end
        end
    elseif mode == "directed"
        for (key,value) in edge_weights
            v = key[1]
            if haskey(weights_count, v)
                weights_count[v] += value
            else
                weights_count[v] = value
            end
        end
    end
    return weights_count
end

In [175]:
function nodes_num_descriptors(graph)
    num_nodes = nv(graph)
    node_descriptors = Dict{Int, Vector}()
    degrees = degree(graph)
    strenghts = compute_strength(edge_weights)
    aspls = shortest_paths(graph).dists
    lspls = maximum(collect(adjacency_matrix(graph)); dims = 2)
    b_centralities = betweenness_centrality(graph) 
    e_centralities = eigenvector_centrality(graph)
    pageranks = pagerank(graph)
    for i in vertices(graph)
        deg = degrees[i]
        stren = strenghts[i]
        aspl = sum(aspls[i,:])/(num_nodes-1)
        lspl = lspls[i]
        cl_cf = local_clustering_coefficient(graph, i)
        bc = b_centralities[i] 
        ec = e_centralities[i]
        pr = pageranks[i]
        
        node_descriptors[i] = [deg,stren, aspl, lspl, cl_cf, bc, ec, pr]
    end
    println(construct_df(node_descriptors, ["Degree", "Strength", "ASPL", "LSPL", "Clust Coeff", "Betweeness", "Eigenvector", "PageRank"]))
end

nodes_num_descriptors (generic function with 1 method)

In [176]:
nodes_num_descriptors(graph)

9×8 DataFrame
 Row │ Degree   Strength  ASPL     LSPL     Clust Coeff  Betweeness  Eigenvector  PageRank  
     │ Float64  Float64   Float64  Float64  Float64      Float64     Float64      Float64   
─────┼──────────────────────────────────────────────────────────────────────────────────────
   1 │     8.0       8.0    1.0        1.0     0.285714   0.571429      0.57735   0.233766
   2 │     3.0       3.0    1.625      1.0     0.666667   0.0178571     0.288675  0.0957793
   3 │     3.0       3.0    1.625      1.0     0.666667   0.0178571     0.288675  0.0957793
   4 │     3.0       3.0    1.625      1.0     0.666667   0.0178571     0.288675  0.0957793
   5 │     3.0       3.0    1.625      1.0     0.666667   0.0178571     0.288675  0.0957793
   6 │     3.0       3.0    1.625      1.0     0.666667   0.0178571     0.288675  0.0957793
   7 │     3.0       3.0    1.625      1.0     0.666667   0.0178571     0.288675  0.0957793
   8 │     3.0       3.0    1.625      1.0     0.666667   0.0178